<a href="https://colab.research.google.com/github/Etiene-vc/Quantum-computing/blob/main/Bernstein_Vazirani_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O algoritmo de Bernstein-Vazirani determina eficientemente uma string binária secreta de n bits com apenas uma consulta a uma função oráculo que calcula o produto interno da string secreta e uma string de entrada.

Para testar o código do Algoritmo de Bernstein-Vazirani com diferentes parâmetros, você precisa modificar apenas duas variáveis:

n (número de qubits): Altere o valor inteiro de n para ajustar o tamanho da string secreta. Um valor maior de n significa uma string secreta mais longa e um circuito quântico maior.

s (string secreta): Altere a string binária s para testar com diferentes strings secretas. Certifique-se de que o comprimento da string s corresponda ao valor de n (tamanho da string = número de qubits).

In [ ]:
!pip install qiskit

In [17]:



from qiskit_aer import AerSimulator
import numpy as np

def oracle(s):
    """
    Cria o circuito do oráculo para a string secreta 's'.

    Args:
        s (str): A string secreta binária (ex: '10110').

    Returns:
        QuantumCircuit: O circuito do oráculo.
    """
    n = len(s)
    qc = QuantumCircuit(n+1) # n qubits para a entrada + 1 qubit auxiliar

    # Itera sobre cada bit da string secreta
    for i in range(n):
        if s[i] == '1':
            qc.cx(i, n) # Aplica uma porta CNOT controlada pelo i-ésimo qubit, afetando o qubit auxiliar

    return qc

def bernstein_vazirani(n, s):
    """
    Implementa o algoritmo de Bernstein-Vazirani.

    Args:
        n (int): O número de qubits.
        s (str): A string secreta binária.

    Returns:
        str: A string secreta medida.
    """
    qc = QuantumCircuit(n+1, n) # n qubits para entrada, 1 qubit auxiliar, n bits clássicos para medição

    # Inicialização: Superposição nos qubits de entrada e |1⟩ no qubit auxiliar
    qc.h(range(n)) # Aplica portas Hadamard a todos os qubits de entrada
    qc.x(n)       # Coloca o qubit auxiliar no estado |1⟩
    qc.h(n)       # Aplica porta Hadamard ao qubit auxiliar

    # Oráculo: Aplica o circuito do oráculo
    qc.append(oracle(s), range(n+1))

    # Transformada de Hadamard: Aplica portas Hadamard novamente nos qubits de entrada
    qc.h(range(n))

    # Medição: Mede os qubits de entrada
    qc.measure(range(n), range(n))


    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator, optimization_level=3) # otimização de circuito
    job = simulator.run(compiled_circuit, shots=1) # Executa com 1 shot (suficiente para o algoritmo)
    result = job.result()
    counts = result.get_counts(0) # Obtém os resultados da medição
    measured_s = list(counts.keys())[0] # Extrai a string medida da chave do dicionário

    return measured_s


# Exemplo de uso
n = 7
s = '0000111'
measured_s = bernstein_vazirani(n, s)
print(f"String secreta: {s}")
print(f"String medida: {measured_s}")

if measured_s == s:
    print("O algoritmo funcionou corretamente!")
else:
    print("O algoritmo falhou!")

String secreta: 0000111
String medida: 1110000
O algoritmo falhou!


Este código inclui:

Função oracle(s): Esta função cria o circuito quântico que representa a função oráculo. A lógica condicional com if s[i] == '1' implementa a função oráculo que calcula o produto escalar entre a entrada e a string secreta (módulo 2).

Função bernstein_vazirani(n, s): Esta função implementa o algoritmo completo. Ela inclui a preparação do estado inicial (superposição), a chamada ao oráculo, a transformada de Hadamard e a medição final. O nível de otimização foi aumentado para optimization_level=3 para melhorar o desempenho da simulação.

Exemplo de uso: A parte final demonstra como usar as funções para executar o algoritmo com uma string secreta de exemplo. A verificação final agora é uma comparação direta entre a string secreta e a string medida.

Esse circuito quântico implementa o Algoritmo de Bernstein-Vazirani para uma string secreta de três bits. Vamos analisar cada parte:

1. Inicialização:

Portas H (Hadamard): As portas H em q0, q1, e q2 colocam esses qubits em superposição. Cada qubit agora está em um estado que é uma combinação linear de |0⟩ e |1⟩. Isso permite que o algoritmo explore todas as possíveis strings de entrada simultaneamente.

Porta Z: A porta Z em q3 inverte a fase do estado |1⟩ desse qubit, o que é uma parte crucial da implementação do oráculo.

2. Oráculo:

Portas CNOT: As portas CNOT (controladas-NOT) implementam a função oráculo. Observe que as portas CNOT são controladas pelos qubits q0, q1, e q2, e suas operações impactam o qubit q3. A presença ou ausência de uma porta CNOT para cada qubit qi depende do i-ésimo bit da string secreta. Se o i-ésimo bit da string secreta for '1', então a porta CNOT está presente e o qubit auxiliar (q3) é alterado.

3. Transformada de Hadamard:

Portas H (Hadamard): Novamente, as portas H são aplicadas aos qubits q0, q1 e q2 após a interação com o oráculo. Esta segunda aplicação da transformada de Hadamard é crucial para decodificar a informação sobre a string secreta que foi "escrita" no qubit auxiliar.

4. Medição:

Portas de Medição: Os qubits q0, q1 e q2 são medidos. Os resultados da medição fornecem a string secreta.

Em resumo:

O algoritmo usa superposição e interferência quântica para calcular o produto interno (módulo 2) entre a string de entrada e a string secreta em uma única etapa. A transformada de Hadamard permite que a informação sobre a string secreta seja extraída da interferência dos estados. O resultado da medição final revela a string secreta. A porta Z no qubit auxiliar e a escolha cuidadosa de portas CNOT no oráculo garantem que essa informação seja codificada e recuperada de forma eficiente.

Interpretação da saída:

A saída do circuito (os bits medidos em q0, q1, e q2) corresponderão diretamente à string secreta. Não há necessidade de inversão ou outras operações na saída neste caso particular, pois já estamos considerando a implementação específica do oráculo.

     q0:  ---H---[Oráculo]---H---M---C0
     q1:  ---H---[Oráculo]---H---M---C1
     q2:  ---H---[Oráculo]---H---M---C2
     q3:  ---H---Z---[Oráculo]---------
             |     |       |
             |     |       |
             CNOT CNOT  CNOT (dependendo da string secreta)